In [ ]:
# label relevancy

In [ ]:
import numpy as np
from scipy.stats import spearmanr

In [ ]:
# cosine similarity
def cosine(a,b):
    return np.dot(a,b)/(np.linalg.norm(a)*np.linalg.norm(b))

In [ ]:
# load-in your training probabilities (alternatively, test probabilities or label embeddings)
prob = np.load('train_prob.npy') 

In [ ]:
# initialize
K = prob.shape[1]
result_re = np.zeros((K,K))

In [ ]:
# calculate label relevancy (whole-hierarchy)
for i in range(K):
    for j in range(K):
        if i<j:
            # if probabilities are used
            result_re[i,j] = result_re[j,i] = spearmanr(prob[:,i],prob[:,j])[0]
            
            # if embeddings are used
            # result_re[i,j] = result_re[j,i] = cosine(prob[:,i],prob[:,j])

In [ ]:
# normalize
result_re = np.abs(result_re)
result_re = result_re/np.sum(result_re,axis=1,keepdims=True)
result_re += np.eye(K)

In [ ]:
# label confusion and sample importance learning

In [ ]:
result_co = np.zeros((K))

In [ ]:
# load in training set probabilities and true labels
prob = np.load('train_prob.npy') 
labels = np.load('train_y.npy')

In [ ]:
# calculate label confusion
for i in range(K):
    for j in range(K):
        index = np.array(labels[:,i]*(1-labels[:,j]), dtype='bool')
        result_co[i,j] = np.sum(prob[index,j])

In [ ]:
# assign sample weights
n = len(labels)
tau = 1.2 # temperature parameter
weights = np.array((n,K))
for i in range(n):
    for j in range(K):
        if labels[i,j]==0:
            weights[i,j]=1
        else:
            index = np.array(1-labels[i,:], dtype='bool')
            temp = np.sum(np.exp(tau * result_co[j,index])-1)
            weights[i,j] = 1+temp